In [1]:
f = open("The_Five_Orange_Pips.txt", "r", encoding='utf8')
whole_text = f.read()
f.close()

In [2]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"
digits = "([0-9])"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    if "..." in text: text = text.replace("...","<prd><prd><prd>")
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences


In [3]:
all_sentence = split_into_sentences(whole_text)
data_frame_column = all_sentence.copy()

In [4]:
#library that contains punctuation
import string

print(type(string.punctuation))

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
for i in range(len(all_sentence)):
    all_sentence[i] = remove_punctuation(all_sentence[i])

<class 'str'>


In [5]:
for i in range(len(all_sentence)):
    all_sentence[i] = all_sentence[i].lower()

In [6]:
def tokenization(text):
    tokens = re.split('\W+',text)
    return tokens
#applying function to the column
for i in range(len(all_sentence)):
    all_sentence[i] = tokenization(all_sentence[i])

In [7]:
#importing nlp library
import nltk
from nltk.corpus import stopwords
#Stop words present in the library
stopwords = stopwords.words('english')

#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

#applying the function
for i in range(len(all_sentence)):
    all_sentence[i] = remove_stopwords(all_sentence[i])

In [8]:
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()

#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
for i in range(len(all_sentence)):
    all_sentence[i] = stemming(all_sentence[i])


In [9]:
from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text
# for i in range(len(all_sentence)):
#     all_sentence[i] = lemmatizer(all_sentence[i])


In [10]:
import numpy as np
import pandas as pd

In [11]:
import itertools
words = list(itertools.chain.from_iterable(all_sentence))
nump_words = np.array(words)
uniq_words = np.unique(nump_words)


In [12]:
text_word = np.zeros((len(uniq_words), len(all_sentence)))

import math

def frequently_words (wor, sent):
    repetition = 0
    for wo in sent:
        if (wo == wor):
            repetition += 1
    return(repetition)

def is_there (wor, all_sent):
    ni = 0
    for sen in all_sent:
        for wo in sen:
            if (wo == wor):
                ni += 1
                break
    return(ni)

row_num = len(uniq_words)
col_num = len(all_sentence)

text_word = np.zeros((row_num, col_num))
for i in range(row_num):
    for j in range(col_num):
        text_word[i,j] = frequently_words(uniq_words[i], all_sentence[j]) * math.log(col_num/ is_there(uniq_words[i], all_sentence))


In [13]:
U, s, Vt = np.linalg.svd(text_word)

In [14]:
print(U[:,0])

[0.0372589  0.00021403 0.00169323 ... 0.05063434 0.00863236 0.00119039]


In [15]:
def display_important_vocabulary (vocabs, scores_arg, num):
    if (num > len(vocabs)):
        print("Error!! Greater than the number of vocabularies")
    else:
        for i in range(1, num+1):
            print('The vocabulary number ' + str(i) + ' is:: ', vocabs[scores_arg[-i]])


In [16]:
display_important_vocabulary(uniq_words, np.argsort(U[:,0]), 50)

The vocabulary number 1 is::  one
The vocabulary number 2 is::  would
The vocabulary number 3 is::  adventur
The vocabulary number 4 is::  upon
The vocabulary number 5 is::  man
The vocabulary number 6 is::  fact
The vocabulary number 7 is::  like
The vocabulary number 8 is::  k
The vocabulary number 9 is::  case
The vocabulary number 10 is::  among
The vocabulary number 11 is::  never
The vocabulary number 12 is::  connect
The vocabulary number 13 is::  singular
The vocabulary number 14 is::  societi
The vocabulary number 15 is::  final
The vocabulary number 16 is::  account
The vocabulary number 17 is::  time
The vocabulary number 18 is::  day
The vocabulary number 19 is::  room
The vocabulary number 20 is::  howev
The vocabulary number 21 is::  furnitur
The vocabulary number 22 is::  british
The vocabulary number 23 is::  warehous
The vocabulary number 24 is::  loss
The vocabulary number 25 is::  lower
The vocabulary number 26 is::  luxuri
The vocabulary number 27 is::  uffa
The voc

In [17]:
U[:,0].shape

(1447,)

In [18]:
Vt[0,:].shape

(466,)

In [19]:
print(Vt[0,:][np.argsort(Vt[0,:])])

[-3.06992634e-18 -3.05825034e-18 -2.26160797e-18 -3.00926554e-36
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  3.81426913e-21
  5.62905005e-21  9.97860727e-21  1.81039295e-20  2.24874013e-19
  2.71991927e-19  8.17292488e-19  1.10593404e-18  1.45089224e-18
  3.42840376e-05  2.04725129e-04  2.20604247e-04  4.43418053e-04
  4.43418053e-04  4.43418053e-04  4.94597519e-04  5.27380975e-04
  5.34793833e-04  5.42267018e-04  5.97014024e-04  6.31696032e-04
  8.00806434e-04  9.59441008e-04  1.03087167e-03  1.03781514e-03
  1.17880627e-03  1.19492583e-03  1.23894585e-03  1.24411177e-03
  1.29179265e-03  1.36670930e-03  1.55242042e-03  1.67318294e-03
  1.72001529e-03  1.75088907e-03  1.79799379e-03  1.89995470e-03
  1.94888081e-03  1.94888081e-03  1.94888081e-03  2.17541774e-03
  2.20056385e-03  2.24194872e-03  2.25411876e-03  2.26267775e-03
  2.36049618e-03  2.39951

In [20]:
np.argsort(Vt[0,:])

array([237, 419, 235,  38,  83, 319,  85, 454, 211, 447, 288, 335, 196,
       197, 415, 317,  43, 207, 203,  18, 384, 459, 233, 231, 423, 450,
        30, 307, 240, 126,  17, 375, 218, 179, 428, 342, 438, 127, 334,
        44, 437, 340, 407, 252, 432, 148, 320, 406, 421, 380, 268, 145,
       208, 341, 143, 440, 245, 386,  39, 273, 215, 316, 416, 403, 318,
       204, 140, 164, 152, 202, 192, 418, 191, 298,  35, 149, 253, 336,
        49, 408, 287, 260, 161, 397, 265, 378, 284, 325, 165, 379, 263,
       229, 238, 315, 185, 374, 226,  92,  40, 261, 402,  55, 230, 189,
       200, 267, 286, 414, 445, 264,  25, 367, 354, 144, 457,  50, 324,
        15, 239, 269, 422, 283, 247,  24, 225, 105, 195, 417, 427, 154,
       163, 186,  51,  37, 139,  91, 300,  16,  87, 282, 159, 213, 446,
        46, 424, 158, 141, 234, 327,  54, 210, 162, 155, 201, 153, 256,
       170,  13, 272, 431, 302, 456, 147, 383, 439, 250, 156,  90, 266,
       212, 190, 257, 434, 124, 271, 259, 160, 349, 157,  19, 31

In [21]:
def display_important_sentences (sentences, scores_arg, num):
    if (num > len(sentences)):
        print("Error!! Greater than the number of sentences")
    else:
        for i in range(1, num+1):
            print('The sentence number ' + str(i) + ' is:')
            print(sentences[scores_arg[-i]])
            print("")


In [22]:
display_important_sentences(data_frame_column, np.argsort(Vt[0,:]), 20)

The sentence number 1 is:
Among my headings under this one twelve months I find an account of the adventure of the Paradol Chamber, of the Amateur Mendicant Society, who held a luxurious club in the lower vault of a furniture warehouse, of the facts connected with the loss of the British bark Sophy Anderson, of the singular adventures of the Grice Patersons in the island of Uffa, and finally of the Camberwell poisoning case.

The sentence number 2 is:
Most of his time he would spend in his room, with the door locked upon the inside, but sometimes he would emerge in a sort of drunken frenzy and would burst out of the house and tear about the garden with a revolver in his hand, screaming out that he was afraid of no man, and that he was not to be cooped up, like a sheep in a pen, by man or devil.

The sentence number 3 is:
All day the wind had screamed and the rain had beaten against the windows, so that even here in the heart of great, hand-made London we were forced to raise our minds 

In [23]:
import numpy as np
from sklearn.decomposition import NMF

def house_holder(x_vector, dim):
    x_vector = list(x_vector)
    x = np.array(x_vector).reshape(-1,1)
    w = x/np.linalg.norm(x)

    e = np.zeros_like(x)
    e[0,0] = 1
    if (x[0,0] != 0):
        v = np.sign(x[0,0]) * e
    else:
        v = e

    small_u = (w - v)/ np.linalg.norm(w - v)

    u = np.zeros((dim,1))
    for i in range(len(small_u)):
        u[-i-1,0] = small_u[-i-1,0]

    i = np.diag(np.ones(dim))
    h = i - (u @ u.T)* 2#/np.linalg.norm(u @ u.T))

    return(h)

def change_col (A, col1, col2):
    a = np.zeros_like(A)
    m, n = A.shape
    for i in range(n):
        if (i != col1 and i != col2):
            a[:,i] = A[:,i]
    a[:,col1] = A[:,col2]
    a[:,col2] = A[:,col1]
    return(a)


data_frame_column_copyy = data_frame_column.copy()
data_frame_column_copyy = np.array(data_frame_column_copyy).reshape(1,-1)
data_frame_column_indexx = range(len(data_frame_column))
data_frame_column_indexx = np.array(data_frame_column_indexx).reshape(1,-1)


def extraction (A, num_sen, data_frame_column_index, data_frame_column_copy):

    m, n = A.shape
    model = NMF(n_components=num_sen+1, init='random', random_state=0)
    W = model.fit_transform(A)
    H = model.components_


    k, n = H.shape

    for i in range(num_sen):
        dict_norms = {}
        for j in range (n):
            dict_norms[j] = np.linalg.norm(H[i:, j])

        max_norm = max(dict_norms.values())
        if (math.isnan(max_norm)):
            print("Error!! This computer cannot calculate more than  "+ str(i) + " sentence")
            break

        for key in dict_norms.keys():
            if (dict_norms[key] == max_norm):
                high_sent = key 

        H = change_col(H, i, high_sent)
        data_frame_column_copy = change_col(data_frame_column_copy, i, high_sent)
        data_frame_column_index = change_col(data_frame_column_index, i, high_sent)
        A = change_col(A, i, high_sent)

        q = house_holder(H[i:,i], k)
        H = q @ H
        W = W @ q

    return (data_frame_column_index[0, :i])



In [24]:
scorelist = extraction(text_word, 11, data_frame_column_indexx, data_frame_column_copyy)
print(scorelist)
scorelist = list(scorelist)
scorelist.reverse()

[295 311 301 355 114 369   5   6   9 130]


C:\Users\Hosein\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\decomposition\_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [25]:
display_important_sentences(data_frame_column, scorelist, 10)

The sentence number 1 is:
To carry the art, however, to its highest pitch, it is necessary that the reasoner should be able to utilize all the facts which have come to his knowledge; and this in itself implies, as you will readily see, a possession of all knowledge, which, even in these days of free education and encyclopaedias, is a somewhat rare accomplishment.

The sentence number 2 is:
His extreme love of solitude in England suggests the idea that he was in fear of someone or something, so we may assume as a working hypothesis that it was fear of someone or something which drove him from America.

The sentence number 3 is:
Botany variable, geology profound as regards the mud-stains from any region within fifty miles of town, chemistry eccentric, anatomy unsystematic, sensational literature and crime records unique, violin-player, boxer, swordsman, lawyer, and self-poisoner by cocaine and tobacco.

The sentence number 4 is:
This     terrible secret society was formed by some ex-Conf